### Load the data

In [1]:
import numpy as np
import pandas as pd

table = pd.read_csv("positionData.csv", low_memory=False)
display(table.head())

,TOPchampId,TOPkeyRune,TOPspell1,TOPspell2,JNGchampId,JNGkeyRune,JNGspell1,JNGspell2,MIDchampId,MIDkeyRune,MIDspell1,MIDspell2,ADCchampId,ADCkeyRune,ADCspell1,ADCspell2,SUPchampId,SUPkeyRune,SUPspell1,SUPspell2
0,19,2,4,5,56,4,4,9,18,3,4,0,118,0,4,7,34,14,4,5
1,0,13,4,0,54,4,4,9,139,7,4,5,47,0,4,7,10,4,4,5
2,43,2,4,0,36,5,6,9,131,8,4,0,64,0,4,0,74,7,4,5
3,93,2,4,0,42,4,4,9,29,4,4,5,21,2,4,7,12,8,4,5
4,58,2,4,0,79,6,4,9,117,8,4,5,45,3,4,7,140,8,4,5


### Prepare the data extracting x and y

In [2]:
import time
import itertools

b = time.time()

perms = np.array(list(itertools.permutations([0, 1, 2, 3, 4])))
y_cac = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]])

x = np.zeros((len(table)//5*120, 20))
y = np.zeros((len(table)//5*120, 25))

for i in range(len(table)//5):
    x_cac = np.array([table.iloc[i,0:4], table.iloc[i,4:8], table.iloc[i,8:12], table.iloc[i,12:16], table.iloc[i,16:20]])
         
    for perm in range(len(perms)):
        
        for e in range(len(perms[perm])):
            x[i*120+perm][4*e:4*(e+1)] = x_cac[perms[perm][e]]
            y[i*120+perm][5*e:5*(e+1)] = y_cac[perms[perm][e]]

print("Execution time:", round(time.time()-b, 2), "seconds")

Execution time: 19.02 seconds


### Shuffle the data and check it is ok

In [3]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

x, y = unison_shuffled_copies(x, y)

print(x[2])
print(y[2])

[ 45.   3.   4.   7. 138.   7.   4.   6. 133.   2.   4.   5.  54.   4.
   4.   9.  69.  14.   4.   0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0.]


### Create the loaders and separate training and testing data

In [4]:
import torch
import torch.utils.data

train_ratio = 0.8
bs = 64

thold = int(len(x)*train_ratio)

train = list(zip(x[:thold], y[:thold]))
trainloader = torch.utils.data.DataLoader(train, batch_size=bs, shuffle=True, num_workers=2)

test = list(zip(x[thold:], y[thold:]))
testloader = torch.utils.data.DataLoader(test, batch_size=bs, shuffle=True, num_workers=2)

### Define the Neural Network

In [5]:
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0")


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.i = torch.arange(5).view(1, 5)#.to(device)
        
        #self.i2 = torch.arange(5).view(5, 1, 1).to(device)
        #self.j2 = torch.arange(bs).view(1, bs, 1).to(device)
        
        self.id = torch.zeros(5, bs, 141)#.to(device)
        self.rune = torch.zeros(5, bs, 17)#.to(device)
        self.spells = torch.zeros(5, bs, 10)#.to(device)
        
        self.idC = nn.Linear(141, 50)
        self.runeC = nn.Linear(17, 9)
        self.spellsC = nn.Linear(10, 5)
        
        self.c = nn.Linear(50+9+5, 50)
        
        self.fc2 = nn.Linear((50)*5, 400)
        self.fc3 = nn.Linear(400, 25)

    def forward(self, x):
        #bTime = time.time()
        
        self.id.zero_()
        self.rune.zero_()
        self.spells.zero_()
        
        print(time.time()-bTime, "clean")
        bTime = time.time()
        
        #for j in range(5):
        #    for i in range(len(x)):
        #        self.id[j][i][int(x[i][j*4+0].item())] = 1
        #        self.rune[j][i][int(x[i][j*4+1].item())] = 1
        #        self.spells[j][i][int(x[i][j*4+2].item())] = 1
        #        self.spells[j][i][int(x[i][j*4+3].item())] = 1
        
        k1 = x[:, self.i*4].long().transpose(0, 2).view(5, bs, 1)
        k2 = x[:, self.i*4+1].long().transpose(0, 2).view(5, bs, 1)
        k3 = x[:, self.i*4+2].long().transpose(0, 2).view(5, bs, 1)
        k4 = x[:, self.i*4+3].long().transpose(0, 2).view(5, bs, 1)
        
        #print(x[:, self.i*4].long().transpose(0, 2))
        
        self.id.scatter_(2, k1, 1)
        self.rune.scatter_(2, k2, 1)
        self.spells.scatter_(2, k3, 1)
        self.spells.scatter_(2, k4, 1)
        
        #print(np.array_equal(self.rune[0][10].numpy(),self.rune.scatter(2, k2, 1)[0][10].numpy()))
        #print(self.rune[0][10].numpy(),self.rune.scatter(2, k2, 1)[0][10].numpy())
        
        #for i in range(5):
        #    for j in range(bs):
        #        self.id[i][j][k1[i][j][0]] = 1
        #        self.rune[i][j][k2[i][j][0]] = 1
        #        self.spells[i][j][k3[i][j][0]] = 1
        #        self.spells[i][j][k4[i][j][0]] = 1
                
        #print(time.time()-bTime, "fill")
        #bTime = time.time()
        
        x = torch.cat((
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[0])), 
                F.relu(self.runeC(self.rune[0])), 
                F.relu(self.spellsC(self.spells[0])),
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[1])), 
                F.relu(self.runeC(self.rune[1])), 
                F.relu(self.spellsC(self.spells[1])),
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[2])), 
                F.relu(self.runeC(self.rune[2])), 
                F.relu(self.spellsC(self.spells[2])),
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[3])), 
                F.relu(self.runeC(self.rune[3])), 
                F.relu(self.spellsC(self.spells[3])),
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[4])), 
                F.relu(self.runeC(self.rune[4])), 
                F.relu(self.spellsC(self.spells[4])),
            ), dim=1))), 
            ), dim=1)
        
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        
        #print(time.time()-bTime, "compute")

        return x

net = Net()
#net.to(device)

### Define a loss function and a optimizer

In [6]:
import torch.optim as optim

#optimizer = optim.SGD(net.parameters(), lr=0.2, momentum=0.9)
optimizer = optim.Adam(net.parameters())

### Train the network

In [7]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0 # mean loss in the 2000 batches between printing and printing
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = F.binary_cross_entropy(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        if i % 1000 == 999:
            lossc = 0
            count = 0
            for data in testloader:
                images, labels = data
                #images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                lossc += F.binary_cross_entropy(outputs, labels.float()).item()
                if count > 10:
                    break
                count += 1
            print('test loss: '+str(round(lossc/count, 4)))

print('Finished Training')

D:\ANACONDA\lib\site-packages\torch\nn\functional.py:1025: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   100] loss: 0.521
[1,   200] loss: 0.382
[1,   300] loss: 0.162
[1,   400] loss: 0.089
[1,   500] loss: 0.076
[1,   600] loss: 0.071
[1,   700] loss: 0.066
[1,   800] loss: 0.065
[1,   900] loss: 0.059
[1,  1000] loss: 0.059
test loss: 0.066
[1,  1100] loss: 0.058
[1,  1200] loss: 0.055
[1,  1300] loss: 0.054
[1,  1400] loss: 0.055
[1,  1500] loss: 0.052
[1,  1600] loss: 0.052
[1,  1700] loss: 0.053
[1,  1800] loss: 0.050
[1,  1900] loss: 0.049
[1,  2000] loss: 0.047
test loss: 0.0535
[1,  2100] loss: 0.047
[1,  2200] loss: 0.047
[1,  2300] loss: 0.047
[1,  2400] loss: 0.044
[1,  2500] loss: 0.044
[1,  2600] loss: 0.043
[1,  2700] loss: 0.043
[1,  2800] loss: 0.044
[1,  2900] loss: 0.045
[1,  3000] loss: 0.046
test loss: 0.045
[1,  3100] loss: 0.040
[1,  3200] loss: 0.042
[1,  3300] loss: 0.039
[1,  3400] loss: 0.044
[1,  3500] loss: 0.041


KeyboardInterrupt: 

In [12]:
lossc = 0
correct0 = 0
correct1 = 0
correct2 = 0
correct3 = 0
correct4 = 0
correct5 = 0
count = 0
for data in testloader:
    images, labels = data
    #images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    
    lossc += F.binary_cross_entropy(outputs, labels.float()).item()
    
    for i in range(len(outputs)):
        ncorrect = 0
        if torch.argmax(outputs[i][:5]) == torch.argmax(labels[i][:5]):
            ncorrect += 1
        if torch.argmax(outputs[i][5:10]) == torch.argmax(labels[i][5:10]):
            ncorrect += 1
        if torch.argmax(outputs[i][10:15]) == torch.argmax(labels[i][10:15]):
            ncorrect += 1
        if torch.argmax(outputs[i][15:20]) == torch.argmax(labels[i][15:20]):
            ncorrect += 1
        if torch.argmax(outputs[i][20:]) == torch.argmax(labels[i][20:]):
            ncorrect += 1
        
        if ncorrect == 0:
            correct0 += 1
        if ncorrect == 1:
            correct1 += 1
        if ncorrect == 2:
            correct2 += 1
        if ncorrect == 3:
            correct3 += 1
        if ncorrect == 4:
            correct4 += 1
        if ncorrect == 5:
            correct5 += 1
        
    if count > 100:
        break
    count += 1
    
print('loss: '+str(round(lossc/count, 5)))
print('5 correct: '+str(round(correct5/(count*bs)*100, 2))+'%')
print('4 correct: '+str(round(correct4/(count*bs)*100, 2))+'%')
print('3 correct: '+str(round(correct3/(count*bs)*100, 2))+'%')
print('2 correct: '+str(round(correct2/(count*bs)*100, 2))+'%')
print('1 correct: '+str(round(correct1/(count*bs)*100, 2))+'%')
print('0 correct: '+str(round(correct0/(count*bs)*100, 2))+'%')

D:\ANACONDA\lib\site-packages\torch\nn\functional.py:1025: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


loss: 0.00342
5 correct: 100.26%
4 correct: 0.56%
3 correct: 0.17%
2 correct: 0.0%
1 correct: 0.0%
0 correct: 0.0%


In [22]:
import pickle

with open('93.59', 'wb') as fp:
    pickle.dump(list(net.parameters()), fp)